### ingest results.json


####1. read results.json

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType, DateType,FloatType

In [0]:
results_schema = StructType(fields=[StructField("resultId", IntegerType(), False),
                                   StructField("raceId", IntegerType(), True),
                                   StructField("driverId", IntegerType(), True),
                                   StructField("constructorId", IntegerType(), True),
                                   StructField("number", IntegerType(), True),
                                   StructField("grid", IntegerType(), True),
                                   StructField("position", IntegerType(), True),
                                   StructField("positionText", StringType(), True),
                                   StructField("positionOrder", IntegerType(), True),
                                   StructField("points", FloatType(), True),
                                   StructField("laps", IntegerType(), True),
                                   StructField("time", StringType(), True),
                                   StructField("milliseconds", IntegerType(), True),
                                   StructField("fastestLap", IntegerType(), True),
                                   StructField("rank", IntegerType(), True),
                                   StructField("fastestLapTime", StringType(), True),
                                   StructField("fastestLapSpeed", FloatType(), True),
                                   StructField("statusId", StringType(), True)])

In [0]:
result_df=spark.read.schema(results_schema).json("abfss://raw@formula1dlbtg.dfs.core.windows.net/results.json")

####2. rename

In [0]:
from pyspark.sql.functions import current_timestamp

In [0]:
results_with_column_df = result_df.withColumnRenamed("resultId", "result_id").withColumnRenamed("raceId", "race_id") \
.withColumnRenamed("driverId", "driver_id").withColumnRenamed("constructorId", "constructor_id") \
.withColumnRenamed("positionText", "position_text").withColumnRenamed("positionOrder", "position_order") \
.withColumnRenamed("fastestLap", "fastest_lap").withColumnRenamed("fastestLapTime", "fastest_lap_time") \
.withColumnRenamed("fastestLapSpeed", "fastest_lap_speed").withColumnRenamed("statusId", "status_id").withColumn("ingestion_date", current_timestamp())
display(results_with_column_df)

####3.drop

In [0]:
from pyspark.sql.functions import col

In [0]:
result_final_df=results_with_column_df.drop(col("status_id"))
display(result_final_df)


In [0]:
result_final_df.write.mode("overwrite").partitionBy("race_id").parquet("abfss://processed@formula1dlbtg.dfs.core.windows.net/results")

In [0]:
display(spark.read.parquet("abfss://processed@formula1dlbtg.dfs.core.windows.net/results"))